In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# Definição da CNN Simples para MNIST
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [3]:
# Classe Particle (PSO)
class Particle:
    def __init__(self, model):
        self.model = model
        self.position = [param.data.clone() for param in model.parameters()]
        self.velocity = [torch.zeros_like(param) for param in model.parameters()]
        self.pbest_position = self.position[:]
        self.pbest_value = float('inf')

    def update_velocity(self, gbest_position, w=0.5, c1=1.5, c2=1.5):
        for i, param in enumerate(self.velocity):
            r1, r2 = torch.rand(1).item(), torch.rand(1).item()
            self.velocity[i] = (w * self.velocity[i] +
                                c1 * r1 * (self.pbest_position[i] - self.position[i]) +
                                c2 * r2 * (gbest_position[i] - self.position[i]))

    def move(self):
        for i, param in enumerate(self.position):
            self.position[i] += self.velocity[i]

In [4]:
# Classe PSO
class ParticleSwarmOptimizer:
    def __init__(self, model, swarm_size=10):
        self.swarm = [Particle(model) for _ in range(swarm_size)]
        self.gbest_position = [param.data.clone() for param in model.parameters()]
        self.gbest_value = float('inf')

    def evaluate(self, model, criterion, data_loader, device):
        model.eval()
        total_loss = 0.0
        with torch.no_grad():
            for inputs, targets in data_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                total_loss += loss.item()
        return total_loss / len(data_loader)

    def optimize(self, model, criterion, data_loader, device):
        for particle in self.swarm:
            model.load_state_dict({name: param for name, param in zip(model.state_dict().keys(), particle.position)})
            fitness = self.evaluate(model, criterion, data_loader, device)
            if fitness < particle.pbest_value:
                particle.pbest_value = fitness
                particle.pbest_position = [param.clone() for param in particle.position]
            if fitness < self.gbest_value:
                self.gbest_value = fitness
                self.gbest_position = [param.clone() for param in particle.position]
        for particle in self.swarm:
            particle.update_velocity(self.gbest_position)
            particle.move()

In [5]:
#Configuração
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=1000, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 478kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.45MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.93MB/s]


In [6]:
# Modelo e Treinamento
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
pso = ParticleSwarmOptimizer(model)

# Treinamento com PSO + SGD
for epoch in range(10):
    print(f'\nEpoch {epoch+1}')
    model.train()
    pso.optimize(model, criterion, trainloader, device)
    model.load_state_dict({name: param for name, param in zip(model.state_dict().keys(), pso.gbest_position)})
    for inputs, targets in trainloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Avaliação no conjunto de teste
    test_loss = 0.0
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    print(f'Test Loss: {test_loss / len(testloader):.4f}, Accuracy: {100. * correct / total:.2f}%')



Epoch 1
Test Loss: 0.0772, Accuracy: 97.57%

Epoch 2
Test Loss: 0.0794, Accuracy: 97.49%

Epoch 3
Test Loss: 0.0910, Accuracy: 97.24%

Epoch 4
Test Loss: 0.0830, Accuracy: 97.27%

Epoch 5
Test Loss: 0.0858, Accuracy: 97.41%

Epoch 6
Test Loss: 0.0768, Accuracy: 97.67%

Epoch 7
Test Loss: 0.0824, Accuracy: 97.57%

Epoch 8
Test Loss: 0.0853, Accuracy: 97.47%

Epoch 9
Test Loss: 0.0782, Accuracy: 97.69%

Epoch 10
Test Loss: 0.0929, Accuracy: 97.36%
